# L1: Information Retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [0]:
import bz2
import pandas as pd

with bz2.open("/content/app-descriptions.json.bz2.json") as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. Data in a DataFrame can be accessed in various ways, including by row and by column. To give an example, the code in the next cell shows rows 200–204:

In [0]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to acess fields from the description column.

In [0]:
df["description"][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any kind of transformation that is applied before a text is vectorized. Here you can restrict yourself to a very simple preprocessing: tokenization, stop word removal, and lemmatization.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure that you read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information that you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you feel necessary.

In [0]:
#!python -m spacy download xx_ent_wiki_sm

In [0]:
df['description'].to_frame()

,description
0,﻿10000000 is a Dungeon Crawling Puzzle RPG Mat...
1,I 1177 Vårdguidens app får du tillgång till 11...
2,Need counting games for kids & drawing for tod...
3,Beautiful and simple music application for tod...
4,A Fun and intuitive numbers game for your baby...
...,...
1609,심심할 때 한번씩 즐기는 고스톱(맞고).\n복잡하고 요란한 화면이 부담스러우셨나요?...
1610,"소지섭, 안젤리나가 선택한 바로 그 게임!\n\n★ 공식카페에서 구글 사전예약 한정..."
1611,[2019년 8월 업데이트]\n- 신규 레전드 카드 추가\n- 메인 화면 이벤트 시...
1612,★ 레전드 등급 추가로 더욱 강력해진 2019 메이저 업데이트 ★\n\n# 대한민국...


In [0]:
#!pip install googletrans


In [0]:
# translate the descriptions 

# from googletrans import Translator
# translator = Translator()

# for row in range(len(df['description'][1:3])) :
#   df['description'][row] = translator.translate(df['description'][row]).text

# df['description']

In [0]:
import spacy
from spacy.lang.en import English
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en')

def preprocess(text):
    # TODO: Replace the next line with your own code.
    doc = nlp(text,disable=["tagger", "parser","ner","textcat"])
    #
    token_list = [word.lemma_ for word in doc if word]
    #
    filtered_sentence =[] 
    for word in token_list:
      lexeme = nlp.vocab[word]
      if not lexeme.is_stop and not lexeme.is_punct and lexeme.is_alpha:
        filtered_sentence.append(word) 
  
    return filtered_sentence

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses given text by tokenizing it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

**Tip:** To speed up the preprocessing, you can disable loading those spaCy components that you do not need, such as the part-of-speech tagger, parser, and named entity recognizer. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

Test your implementation by running the following cell:

In [0]:
preprocess("Apple is looking at buying U.K. startup for $1 billion")

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


['Apple', 'look', 'buy', 'startup', 'billion']

## Problem 2: Vectorizing

Your next task is to vectorize the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectorizer.

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
# TODO: Replace the next line with your own code.

corpus = df['description']

vectorizer = TfidfVectorizer(tokenizer=preprocess)
X = vectorizer.fit_transform(corpus)


Test your implementation by running the following cell:

In [0]:
X.shape

(1614, 20606)

This should show the dimensions of the matrix `X` to be 1614 × 20651.

## Problem 3: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms have the lowest/highest idf with respect to the app descriptions.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [0]:
# TODO: Replace the next line with your own code.

idf = vectorizer.idf_

In [0]:
#terms_dict = dict(zip(vectorizer.get_feature_names(), idf))

#terms = sorted(terms_dict.items(), key=lambda x: x[1],reverse=True)

# print last 10
#terms[-10:]

# print first 10
#terms[:10]

In [0]:
pdframe = pd.DataFrame({'names':vectorizer.get_feature_names(),'tfidf':idf})

sorted_pd = pdframe.sort_values(by=['tfidf','names'],ascending=[False,True])

In [0]:
# last ten
sorted_pd.iloc[-10:]

,names,tfidf
5661,fun,1.859132
666,app,1.838871
15381,time,1.788581
17057,world,1.781792
16212,use,1.710007
10170,new,1.665665
5564,free,1.584695
5176,feature,1.472359
11448,play,1.412611
5811,game,1.296180


In [0]:
# first 10
sorted_pd[:10]

,names,tfidf
1,aa,7.693943
4,aagbi,7.693943
5,aang,7.693943
6,aat,7.693943
9,abbiamo,7.693943
10,abbott,7.693943
11,abby,7.693943
13,abcd,7.693943
14,abcmouse,7.693943
16,abcsong,7.693943


Now, print out the 10 terms at the top/end of the sorted list:

## Problem 4: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to do solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning, and allows you to easily find a predefined number of app descriptions whose vector representations are closest in distance to the query vector.

In [0]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(metric='cosine')
neigh.fit(X)

def search(query):
    # TODO: Replace the next line with your own code.
    query = vectorizer.transform([query])
    indx = neigh.kneighbors(query, 10, return_distance=False)[0]
    
    return df.iloc[indx]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

Test your implementation by running the following cell:

In [0]:
search("dodge trains")

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
1428,Train Conductor World,Master and manage the chaos of international r...
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
1077,Polar Flow – Sync & Analyze,"Polar Flow is a sports, fitness, and activity ..."
1286,"Strava: Track Running, Cycling & Swimming",Track your fitness with Strava activity tracke...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...


The top hit in the list should be *Subway Surfers*.

## Problem 5: Keyword extraction

A simple method for extracting salient keywords from a document is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a given text.

In [0]:

def keywords(text, n=10):
    # TODO: Replace the next line with your own code.

    v = vectorizer.transform([text])

    pd_frame = pd.DataFrame(v.transpose())
    
    sorted_pd_frame = pd_frame.sort_values(by=[0],ascending=False)
    
    indexes=  sorted_pd_frame[:n].index.tolist()

    n_keywords = [vectorizer.get_feature_names()[i] for i in indexes]
    
    return n_keywords

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

Test your implementation by running the following cell:

In [0]:
print(keywords(df["description"][1428]))
#['train', 'railroad', 'railway', 'rail', 'chaos', 'crash', 'tram', 'haul', 'overcast', 'railyard']

['train', 'railroad', 'railway', 'rail', 'chaos', 'crash', 'timetable', 'overcast', 'railyard', 'locomotive']


This should give the following output:

['train', 'railroad', 'railway', 'rail', 'chaos', 'crash', 'tram', 'haul', 'overcast', 'railyard']

<div class="alert alert-info">
    Please read the General information section on the lab web page before submitting this notebook!
</div>